# LightGBM + ハイパーパラメータチューニング

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import datetime
import seaborn as sns
from sklearn.preprocessing import StandardScaler

import os
for dirname, _, filenames in os.walk('../data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../data/sample_submit.csv
../data/submission_lightBGM.csv
../data/submission_Logistic.csv
../data/test.tsv
../data/train.tsv
../data/.ipynb_checkpoints\sample_submit-checkpoint.csv
../data/.ipynb_checkpoints\submission_lightBGM-checkpoint.csv
../data/.ipynb_checkpoints\submission_Logistic-checkpoint.csv
../data/.ipynb_checkpoints\test-checkpoint.tsv
../data/.ipynb_checkpoints\train-checkpoint.tsv


In [2]:
train_df = pd.read_table('../data/train.tsv', index_col='Date', parse_dates=True)
test_df = pd.read_table('../data/test.tsv', index_col='Date', parse_dates=True)
sample_sub = pd.read_csv('../data/sample_submit.csv')
print(sample_sub)

# set type label
train_df['type'] = 'train'
test_df['type'] = 'test'

# all data
all_df = pd.concat([train_df, test_df], axis=0)

      1267  0.0
0     1268  0.0
1     1269  0.0
2     1270  1.0
3     1271  1.0
4     1272  1.0
...    ...  ...
1261  2529  1.0
1262  2530  1.0
1263  2531  0.0
1264  2532  1.0
1265  2533  0.0

[1266 rows x 2 columns]


## 特徴量前処理

In [3]:
import re

def eda(all_df):
    # データの追加,気温・風速の標準偏差
    #1時間ごとの気温・風速を取得
    T_data = all_df[['T0', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T10', 'T11', 'T12', 'T13', 'T14', 'T15', 'T16', 'T17', 'T18', 'T19', 'T20', 'T21', 'T22', 'T23']]
    WSR_data = all_df[['WSR0', 'WSR1', 'WSR2', 'WSR3', 'WSR4', 'WSR5', 'WSR6', 'WSR7', 'WSR8', 'WSR9', 'WSR10', 'WSR11', 'WSR12', 'WSR13', 'WSR14', 'WSR15', 'WSR16', 'WSR17', 'WSR18', 'WSR19', 'WSR20', 'WSR21', 'WSR22', 'WSR23']]
    # 行ごとの標準偏差を追加
    all_df['T_SD'] = T_data.std(axis=1)
    all_df['WSR_SD'] = WSR_data.std(axis=1)
    """
    # データの削除, T0~T23
    all_df = all_df.drop(columns=['T0', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T10', 'T11', 'T12', 'T13', 'T14', 'T15', 'T16', 'T17', 'T18', 'T19', 'T20', 'T21', 'T22', 'T23'])
    # データの削除, WSR0~WSR23
    all_df = all_df.drop(columns=['WSR0', 'WSR1', 'WSR2', 'WSR3', 'WSR4', 'WSR5', 'WSR6', 'WSR7', 'WSR8', 'WSR9', 'WSR10', 'WSR11', 'WSR12', 'WSR13', 'WSR14', 'WSR15', 'WSR16', 'WSR17', 'WSR18', 'WSR19', 'WSR20', 'WSR21', 'WSR22', 'WSR23'])
    """
    # データの削除, SLP_
    all_df = all_df.drop(columns=['SLP_'])
    return all_df


# 特徴量の削除/追加
all_df = eda(all_df)

# trainとtestに分けなおす
train_df = all_df[all_df['type'] == 'train']
test_df = all_df[all_df['type'] == 'test']
# train正解ラベル
y = train_df['OZONE']

# 学習に不要な特徴量を削除
train_df = train_df.drop(columns=['id', 'OZONE', 'type'])
test_df = test_df.drop(columns=['id', 'OZONE', 'type'])

# 欠損値を平均値で補完
train_df = train_df.fillna(train_df.mean())
test_df = test_df.fillna(test_df.mean())


"""
# データ標準化(rightGBMのときはいらない)
scaler = StandardScaler()
train_df = pd.DataFrame(scaler.fit_transform(train_df), index = train_df.index, columns = train_df.columns)
test_df = pd.DataFrame(scaler.transform(test_df), index = test_df.index, columns = test_df.columns)
"""

print(f'train_df : \n{train_df}\n')
print(f'y : \n{y}\n')

train_df : 
            WSR0  WSR1  WSR2  WSR3  WSR4  WSR5  WSR6  WSR7  WSR8  WSR9  ...  \
Date                                                                    ...   
1998-01-01   0.8   1.8   2.4   2.1   2.0   2.1   1.5   1.7   1.9   2.3  ...   
1998-01-02   2.8   3.2   3.3   2.7   3.3   3.2   2.9   2.8   3.1   3.4  ...   
1998-01-03   2.9   2.8   2.6   2.1   2.2   2.5   2.5   2.7   2.2   2.5  ...   
1998-01-04   4.7   3.8   3.7   3.8   2.9   3.1   2.8   2.5   2.4   3.1  ...   
1998-01-05   2.6   2.1   1.6   1.4   0.9   1.5   1.2   1.4   1.3   1.4  ...   
...          ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
2001-06-29   0.8   0.9   0.3   0.3   0.7   0.8   1.3   1.9   3.0   3.0  ...   
2001-06-30   1.0   0.7   0.8   0.6   0.9   1.2   1.3   1.7   2.8   2.9  ...   
2001-07-01   0.7   1.0   1.4   0.9   1.0   1.2   1.4   1.4   1.1   2.0  ...   
2001-07-02   1.7   1.7   1.1   1.3   1.0   1.0   1.2   1.8   1.8   1.8  ...   
2001-07-03   0.5   0.5   0.8   0.6   0.8

### lightGBM
#### +ハイパーパラメータチューニング(深さ、葉数、学習率)

In [ ]:
from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn.metrics import accuracy_score
import optuna
optuna.logging.disable_default_handler() 

# indexをDateから普通のindexに直す(kholdが使えないため)、日付は消す
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

def objective(trial):
    #khold
    cv = KFold(n_splits=5, random_state=0, shuffle=True)

    # train_acc_list = []
    val_acc_list = []
    # models = []

    # ハイパーパラメータを定義　
    lgb_params = {
        "objective":"binary",
        "metric": "binary_error",
        "force_row_wise" : True,
        "seed" : 0,
        'learning_rate': 0.1,
        'min_data_in_leaf': 5,

        # optunaでパラメータ探索する区間を指定
        #木の最大深度
        "max_depth": trial.suggest_int('max_depth', 3, 20),
        #一本の木の最大葉数
        "num_leaves": trial.suggest_int('num_leaves', 21, 41),
        #学習率
        "learning_rate": trial.suggest_uniform('learning_rate', 0.1, 0.2),
        }

    # KFold で学習させる
    for i ,(trn_index, val_index) in enumerate(cv.split(train_df, y)):

        print(f'Fold : {i}')
        X_train ,X_val = train_df.loc[trn_index], train_df.loc[val_index]
        y_train ,y_val = y[trn_index],y[val_index]

        # LigthGBM用のデータセットを定義
        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_valid = lgb.Dataset(X_val, y_val)

        # model = lgb.train(params = lgb_params, train_set = lgb_train, valid_sets = [lgb_train, lgb_valid], callbacks=[lgb.log_evaluation(period=100), lgb.early_stopping(10)],)
        model = lgb.train(params = lgb_params, train_set = lgb_train, valid_sets = [lgb_train, lgb_valid], callbacks=[lgb.log_evaluation(period=1), lgb.early_stopping(10)],)

        y_pred = model.predict(X_train)

        """
        train_acc = accuracy_score(
            y_train, np.where(y_pred>=0.5, 1, 0)
            )

        train_acc_list.append(train_acc)
        """
        y_pred_val = model.predict(X_val)
        val_acc = accuracy_score(
            y_val, np.where(y_pred_val>=0.5, 1, 0)
            )
        
        val_acc_list.append(val_acc)

     # 最適化の指標にする値を返り値に設定
    return np.mean(val_acc_list)
    
# 最適化タスクを定義
optuna.logging.disable_default_handler()
# 引数のdirectionには最適化する指標を最大化したいのか、最小化したいのかを指定
study = optuna.create_study(direction="maximize")

# 最適化を実行します。この時の探索の試行回数をn_trialsで指定
study.optimize(objective, n_trials=100)

# 最適なパラメータとその時の精度を表示
print("best_value", study.best_value)
print("best_params", study.best_params)

# 最適化の過程を表示 & 可視化
optuna_log_df = study.trials_dataframe(attrs=("number", "value", "params"))
# display(optuna_log_df)

# max_depthと正答率の関係を可視化
plt.scatter(optuna_log_df["params_max_depth"], optuna_log_df["value"])

## 探索したハイパーパラメータでLightGBMを使ってみる

In [31]:
from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn.metrics import accuracy_score

#khold
cv = KFold(n_splits=5, random_state=0, shuffle=True)

train_acc_list = []
val_acc_list = []
models = []

# ハイパーパラメータを定義　
lgb_params = {
    "objective":"binary",
    "metric": "binary_error",
    "force_row_wise" : True,
    "seed" : 0,
    'learning_rate': 0.12231958989629002,
    'min_data_in_leaf': 5,
    'num_leaves': 38,
    'max_depth': 9,
    }

# indexをDateから普通のindexに直す(kholdが使えないため)、日付は消す
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

# KFold で学習させる
for i ,(trn_index, val_index) in enumerate(cv.split(train_df, y)):
    
    print(f'Fold : {i}')
    X_train ,X_val = train_df.loc[trn_index], train_df.loc[val_index]
    y_train ,y_val = y[trn_index],y[val_index]
    
    # LigthGBM用のデータセットを定義
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_valid = lgb.Dataset(X_val, y_val)
    
    model = lgb.train(params = lgb_params, train_set = lgb_train, valid_sets = [lgb_train, lgb_valid], callbacks=[lgb.log_evaluation(period=100), lgb.early_stopping(10)],
                     )
    
    y_pred = model.predict(X_train)
    print(f'y_pred.sum------------------------{np.where(y_pred>=0.5, 1, 0).sum()}')
    print(f'y_train.sum----------------------{y_train.sum()}')
    
    train_acc = accuracy_score(
        y_train, np.where(y_pred>=0.5, 1, 0)
        )
    print(train_acc)
    train_acc_list.append(train_acc)
    
    y_pred_val = model.predict(X_val)
    val_acc = accuracy_score(
        y_val, np.where(y_pred_val>=0.5, 1, 0)
        )
    print(val_acc)
    val_acc_list.append(val_acc)
    
    models.append(model)

print('-'*10 + 'Result' +'-'*10)
print(f'Train_acc : {train_acc_list} , Ave : {np.mean(train_acc_list)}')
print(f'Valid_acc : {val_acc_list} , Ave : {np.mean(val_acc_list)}')

Fold : 0
[LightGBM] [Info] Number of positive: 95, number of negative: 918
[LightGBM] [Info] Total Bins 11371
[LightGBM] [Info] Number of data points in the train set: 1013, number of used features: 73
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.093781 -> initscore=-2.268320
[LightGBM] [Info] Start training from score -2.268320
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[17]	training's binary_error: 0	valid_1's binary_error: 0.0354331
y_pred.sum------------------------95
y_train.sum----------------------95.0
1.0
0.9645669291338582
Fold : 1
[LightGBM] [Info] Number of positive: 83, number of negative: 930
[LightGBM] [Info] Total Bins 11429
[LightGBM] [Info] Number of data points in the train set: 1013, number of used features: 73
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.081935 -> initscore=-2.416344
[LightGBM] [Info] Start training from score -2.416344
Training until validation scores don't improve for 10 rounds
Early s

In [32]:
# モデルを用いてテストデータを予測
test_pred = np.zeros((len(test_df), 5))
for fold_index , gbm in enumerate(models):
    pred_test = gbm.predict(test_df)
    print(np.where(pred_test>=0.5, 1, 0).sum())
    test_pred[:, fold_index] = pred_test
    # test_pred[:, fold_index] = np.where(pred_test>=0.5, 1, 0)

# 行数で繰り返し予測値を代入
test_pred = (np.mean(test_pred, axis=1) >= 0.5).astype(int)
print(f'test_pred.sum : {test_pred.sum()}')
for index, row in sample_sub.iterrows():
    sample_sub.iloc[index,1] = test_pred[index]
    
# 結果を保存
sample_sub.to_csv("../data/submission_lightBGM2.csv", index=False)

18
12
25
25
21
test_pred.sum : 14


### kholdの分割数を5→3にしてみる

In [ ]:
from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn.metrics import accuracy_score
import optuna
optuna.logging.disable_default_handler() 

# indexをDateから普通のindexに直す(kholdが使えないため)、日付は消す
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

def objective(trial):
    #khold
    cv = KFold(n_splits=3, random_state=0, shuffle=True)

    # train_acc_list = []
    val_acc_list = []
    # models = []

    # ハイパーパラメータを定義　
    lgb_params = {
        "objective":"binary",
        "metric": "binary_error",
        "force_row_wise" : True,
        "seed" : 0,
        'learning_rate': 0.1,
        'min_data_in_leaf': 5,

        # optunaでパラメータ探索する区間を指定
        #木の最大深度
        "max_depth": trial.suggest_int('max_depth', 3, 20),
        #一本の木の最大葉数
        "num_leaves": trial.suggest_int('num_leaves', 21, 41),
        #学習率
        "learning_rate": trial.suggest_uniform('learning_rate', 0.1, 0.2),
        }

    # KFold で学習させる
    for i ,(trn_index, val_index) in enumerate(cv.split(train_df, y)):

        print(f'Fold : {i}')
        X_train ,X_val = train_df.loc[trn_index], train_df.loc[val_index]
        y_train ,y_val = y[trn_index],y[val_index]

        # LigthGBM用のデータセットを定義
        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_valid = lgb.Dataset(X_val, y_val)

        # model = lgb.train(params = lgb_params, train_set = lgb_train, valid_sets = [lgb_train, lgb_valid], callbacks=[lgb.log_evaluation(period=100), lgb.early_stopping(10)],)
        model = lgb.train(params = lgb_params, train_set = lgb_train, valid_sets = [lgb_train, lgb_valid], callbacks=[lgb.log_evaluation(period=1), lgb.early_stopping(10)],)

        y_pred = model.predict(X_train)

        """
        train_acc = accuracy_score(
            y_train, np.where(y_pred>=0.5, 1, 0)
            )

        train_acc_list.append(train_acc)
        """
        y_pred_val = model.predict(X_val)
        val_acc = accuracy_score(
            y_val, np.where(y_pred_val>=0.5, 1, 0)
            )
        
        val_acc_list.append(val_acc)

     # 最適化の指標にする値を返り値に設定
    return np.mean(val_acc_list)
    
# 最適化タスクを定義
optuna.logging.disable_default_handler()
# 引数のdirectionには最適化する指標を最大化したいのか、最小化したいのかを指定
study = optuna.create_study(direction="maximize")

# 最適化を実行します。この時の探索の試行回数をn_trialsで指定
study.optimize(objective, n_trials=100)

# 最適なパラメータとその時の精度を表示
print("best_value", study.best_value)
print("best_params", study.best_params)

# 最適化の過程を表示 & 可視化
optuna_log_df = study.trials_dataframe(attrs=("number", "value", "params"))
# display(optuna_log_df)

# max_depthと正答率の関係を可視化
plt.scatter(optuna_log_df["params_max_depth"], optuna_log_df["value"])

In [26]:
from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn.metrics import accuracy_score

#khold
cv = KFold(n_splits=3, random_state=0, shuffle=True)

train_acc_list = []
val_acc_list = []
models = []

# ハイパーパラメータを定義　
lgb_params = {
    "objective":"binary",
    "metric": "binary_error",
    "force_row_wise" : True,
    "seed" : 0,
    'learning_rate': 0.14453401736710864,
    'min_data_in_leaf': 5,
    'num_leaves': 29,
    'max_depth': 17,
    }

# indexをDateから普通のindexに直す(kholdが使えないため)、日付は消す
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

# KFold で学習させる
for i ,(trn_index, val_index) in enumerate(cv.split(train_df, y)):
    
    print(f'Fold : {i}')
    X_train ,X_val = train_df.loc[trn_index], train_df.loc[val_index]
    y_train ,y_val = y[trn_index],y[val_index]
    
    # LigthGBM用のデータセットを定義
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_valid = lgb.Dataset(X_val, y_val)
    
    model = lgb.train(params = lgb_params, train_set = lgb_train, valid_sets = [lgb_train, lgb_valid], callbacks=[lgb.log_evaluation(period=100), lgb.early_stopping(10)],
                     )
    
    y_pred = model.predict(X_train)
    print(f'y_pred.sum------------------------{np.where(y_pred>=0.5, 1, 0).sum()}')
    print(f'y_train.sum----------------------{y_train.sum()}')
    
    train_acc = accuracy_score(
        y_train, np.where(y_pred>=0.5, 1, 0)
        )
    print(train_acc)
    train_acc_list.append(train_acc)
    
    y_pred_val = model.predict(X_val)
    val_acc = accuracy_score(
        y_val, np.where(y_pred_val>=0.5, 1, 0)
        )
    print(val_acc)
    val_acc_list.append(val_acc)
    
    models.append(model)

print('-'*10 + 'Result' +'-'*10)
print(f'Train_acc : {train_acc_list} , Ave : {np.mean(train_acc_list)}')
print(f'Valid_acc : {val_acc_list} , Ave : {np.mean(val_acc_list)}')

Fold : 0
[LightGBM] [Info] Number of positive: 78, number of negative: 766
[LightGBM] [Info] Total Bins 10752
[LightGBM] [Info] Number of data points in the train set: 844, number of used features: 73
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.092417 -> initscore=-2.284473
[LightGBM] [Info] Start training from score -2.284473
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[10]	training's binary_error: 0.00118483	valid_1's binary_error: 0.0638298
y_pred.sum------------------------77
y_train.sum----------------------78.0
0.9988151658767772
0.9361702127659575
Fold : 1
[LightGBM] [Info] Number of positive: 71, number of negative: 774
[LightGBM] [Info] Total Bins 10806
[LightGBM] [Info] Number of data points in the train set: 845, number of used features: 73
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.084024 -> initscore=-2.388892
[LightGBM] [Info] Start training from score -2.388892
Training until validation scores don't improve

In [28]:
# モデルを用いてテストデータを予測
test_pred = np.zeros((len(test_df), 3))
for fold_index , gbm in enumerate(models):
    pred_test = gbm.predict(test_df)
    print(np.where(pred_test>=0.5, 1, 0).sum())
    test_pred[:, fold_index] = pred_test
    # test_pred[:, fold_index] = np.where(pred_test>=0.5, 1, 0)

# 行数で繰り返し予測値を代入
test_pred = (np.mean(test_pred, axis=1) >= 0.5).astype(int)
print(f'test_pred.sum : {test_pred.sum()}')
for index, row in sample_sub.iterrows():
    sample_sub.iloc[index,1] = test_pred[index]
    
# 結果を保存
sample_sub.to_csv("../data/submission_lightBGM2_2.csv", index=False)

23
27
32
test_pred.sum : 22
